# SIL Multi Lingual huge database

### Importing Libraries 

In [14]:
import pandas as pd
import numpy as np
import string
import os 
import re

# Panlex

In [7]:
#Mapping the input folder 
os.chdir(r"C:\Users\Nikhi\OneDrive - purdue.edu\Mod 3\Industry Practicum\Files")

In [8]:
#Opening the required file 
with open('swadesh207.txt',encoding="utf8") as f:
    #Reading the contents
    contents = f.read()
    print(contents)

I
thou/you
he/she
we
ye/you/y'all
they
this
that
here
there
who/who?
what
where/where?
when/when?
how/how?
not
all/every
many/much/multiple/several
some
few/little
other
one/single/that
grind/two
three
four
five
big/bigness/large
long
broad/wide/wideness
fat/thick
dull/heavy/serious
little/small/smallness
short
narrow
thin
female/woman
male/man
human/man/person
child
wife
husband
mother
father
animal
fish
bird
dog
louse
snake
earthworm/worm
tree
forest/woods
branch/stick
fruit
seed
leaf
root
bark
flower
grass
cord/rope
hide/skin
flesh/meat
blood
bone
fat/grease
egg
horn
tail
feather
hair
head
ear
eye
nose
mouth
tooth
tongue
claw/fingernail/nail
foot
leg
knee
arm/hand
wing
abdomen/belly
gut/guts/insides/intestine/intestines
neck
back
breast/breasts
heart
liver
drink
eat
bite
suck
spit
vomit
blow
breath/breathe
laugh/laughter
see
hear
knowledgeable/know
cogitate/reflect/think
smell/sniff
afraid/fear/fright
sleep
alive/life/live/living
dead/die
kill
fight
hunt
beat/hit/strike
cut/hack
spl

In [9]:
#Splitting the words and inputing them into a list 
eng_list=contents.split("\n")

In [15]:
#Creating a dictionary of index and words mapped
dict_eng={}
i=0
for y in eng_list:
    #Since the length of words are 207
    if i<207:
        #incrementing to start the index from zero
        i=i+1
        #Splitting the words and keep them in list
        x=re.split("/",y) 
        #Assigning the list to key (index)
        dict_eng[i]=x
    else:
        break
print(dict_eng)

{1: ['I'], 2: ['thou', 'you'], 3: ['he', 'she'], 4: ['we'], 5: ['ye', 'you', "y'all"], 6: ['they'], 7: ['this'], 8: ['that'], 9: ['here'], 10: ['there'], 11: ['who', 'who?'], 12: ['what'], 13: ['where', 'where?'], 14: ['when', 'when?'], 15: ['how', 'how?'], 16: ['not'], 17: ['all', 'every'], 18: ['many', 'much', 'multiple', 'several'], 19: ['some'], 20: ['few', 'little'], 21: ['other'], 22: ['one', 'single', 'that'], 23: ['grind', 'two'], 24: ['three'], 25: ['four'], 26: ['five'], 27: ['big', 'bigness', 'large'], 28: ['long'], 29: ['broad', 'wide', 'wideness'], 30: ['fat', 'thick'], 31: ['dull', 'heavy', 'serious'], 32: ['little', 'small', 'smallness'], 33: ['short'], 34: ['narrow'], 35: ['thin'], 36: ['female', 'woman'], 37: ['male', 'man'], 38: ['human', 'man', 'person'], 39: ['child'], 40: ['wife'], 41: ['husband'], 42: ['mother'], 43: ['father'], 44: ['animal'], 45: ['fish'], 46: ['bird'], 47: ['dog'], 48: ['louse'], 49: ['snake'], 50: ['earthworm', 'worm'], 51: ['tree'], 52: ['for

In [11]:
#Creating a Panalex dataframe from above dictionary
word_df = pd.DataFrame(columns=['index','word'])
#Looping in the dictionary 
for key,value in dict_eng.items():
    #Since value is a list
    for word in value:
        word_df = word_df.append({'index':key, 'word':word}, ignore_index=True)

In [12]:
#Shape of the dataframe 
word_df.shape

(315, 2)

In [13]:
#Dropping duplicates if any 
word_df.drop_duplicates(inplace=True)
word_df.shape

(315, 2)

### Importing the semantic domains file 

In [17]:
#semantic domain file 
semdom = pd.read_csv('semdom_words_eng.csv')
semdom.head()
#Copying the dataset 
df=semdom.copy()
#Dropping the duplicates 
df.drop_duplicates(subset=['word','cids'],inplace=True)
df.shape

(39854, 3)

## Mapping Semantic domains to Panalex

In [22]:
#Performed left join to ensure words which are not mapped are also there in the file
merged_data = pd.merge(word_df,df, on='word', how='left')

In [23]:
print(merged_data.shape)
merged_data.columns

(1181, 4)


Index(['index', 'word', 'category', 'cids'], dtype='object')

In [24]:
#Count of unique words in the dataset
count_value=merged_data['word'].value_counts()
#count_value.to_csv("CountofWords.csv")

In [25]:
merged_data.head()

,index,word,category,cids
0,1,I,Pronouns,9.2.3
1,2,thou,NaN,NaN
2,2,you,Pronouns,9.2.3
3,2,you,Prompters of attention,9.6.3.3
4,3,he,Pronouns,9.2.3


In [26]:
merged_data1 = merged_data[merged_data['index'].notna()]
merged_data1.shape
merged_data1.head()

,index,word,category,cids
0,1,I,Pronouns,9.2.3
1,2,thou,NaN,NaN
2,2,you,Pronouns,9.2.3
3,2,you,Prompters of attention,9.6.3.3
4,3,he,Pronouns,9.2.3


In [27]:
#Count of unqiue words
count_values2=merged_data1['word'].value_counts()
#count_values2.to_csv("Countofvvalues.csv")

In [28]:
#Words that are not merged 
unmerged_words=merged_data1[merged_data1['cids'].isnull()]['word']
unmerged_words

1             thou
7               ye
10           y'all
24            who?
31          where?
36           when?
44            how?
94         bigness
115       wideness
158      smallness
188          human
420        insides
422     intestines
439        breasts
515           fear
623         fly v.
692          grasp
869           snow
934      whiteness
1118           ner
Name: word, dtype: object

## Implementation of Word2Vec to map the unmapped words

In [63]:
from gensim.models import KeyedVectors
filename = r"C:\Users\Nikhi\Desktop\Files\SIL\GoogleNews-vectors-negative300.bin.gz"
model = KeyedVectors.load_word2vec_format(filename, binary=True)

### Finding the top 10 similar words using word2vec

In [84]:
#creating an empty list
list_of_words = []
#Empty dictionary
dict_similar={}
#lopping each and every row and create most 10 simlar words to the english words using word2vec
for i in range(len(unmerged_words['word'])):
    try:
        #finding the top 10 similar words
        s=model.similar_by_word(unmerged_words.loc[i,'word'], topn=10)
        #finding the similarity scores also
        dict_similar[unmerged_words.loc[i,'word']]=s
    except:
        list_of_words.append(unmerged_words.loc[i,'word'])
#found similar words in the dictionary
print(dict_similar)
#list of unmapped words after word2vec 
print(list_of_words)

{'thou': [('Thou', 0.6597303748130798), ('wherefore_art', 0.6211819052696228), ('thou_Romeo', 0.6205939054489136), ('thy', 0.6048526167869568), ('Thou_art', 0.5877188444137573), ('knowest', 0.5758250951766968), ('thou_attitude', 0.5644581317901611), ('Holier', 0.559604287147522), ('wilt_thou', 0.537873387336731), ('thee', 0.5369542837142944)], 'ye': [('thou_wilt', 0.6854915022850037), ('thy', 0.6826894879341125), ('thee', 0.6740559339523315), ('ye_shall', 0.6575972437858582), ('O_ye', 0.6488433480262756), ('unto_thee', 0.6486373543739319), ('dost_thou', 0.6341127753257751), ('thou_mayest', 0.6289900541305542), ('wilt_thou', 0.6270899176597595), ('saith_unto', 0.6187568306922913)], "y'all": [("ya'll", 0.8755648732185364), ("Y'all", 0.7741464972496033), ("Ya'll", 0.7522830367088318), ('yall', 0.7104735970497131), ('stuntin', 0.6864768862724304), ('move_Bibby_growled', 0.6676869988441467), ('ya', 0.6676328182220459), ('Ding_dang', 0.660785973072052), ('gonna', 0.6550487875938416), ('wanna

In [220]:
#domain dictionary
domain_dict={}
#closest word mapping 
mean_dict={}
#lopping in the dictionary
for i,k in dict_similar.items():
    #since k is a list
    for j in k: 
        #Comparing if the words are already there in sematic domain file
        for wd in df['word']:
            #if yes, then find the cid and break the loop
            if wd==j[0]:
                z=0
                if z<=1:
                    domain_dict[i]=list(df[df['word']==j[0]]['cids'])
                    mean_dict[i]=j[0]
                    z=z+1
                else:
                    break
            else:
                continue
print(domain_dict)
print(mean_dict)

{"y'all": ['9.4.2'], 'smallness': ['8.2.6.2'], 'human': ['2', '3.1'], 'insides': ['8.6.4'], 'intestines': ['2.1.8'], 'breasts': ['2.1.8.3', '2.1.8.4'], 'fear': ['3.4.2.4'], 'grasp': ['7.3.4.4', '7.5.2.2', '8.6'], 'snow': ['1.1.3.4'], 'whiteness': ['1.6.3.1', '2.1.1.1', '2.1.1.5', '2.1.4', '4.1.2', '4.1.9.9', '4.2.6.1.2', '8.3.3.3.1']}
{"y'all": 'gonna', 'smallness': 'nearness', 'human': 'humanity', 'insides': 'innards', 'intestines': 'trachea', 'breasts': 'genitals', 'fear': 'afraid', 'grasp': 'grip', 'snow': 'sleet', 'whiteness': 'white'}


In [222]:
#Converting the final dictionary into dataframe 
word_df1 = pd.DataFrame(columns=['word','cids1'])
#Looping in the dictionary 
for key,value in domain_dict.items():
    #Converting the dictionary into dataframe
    for word in value:
        word_df1 = word_df1.append({'word':key, 'cids1':word}, ignore_index=True)
#printing shape of unmapped words
print(word_df1.shape)
word_df1

(21, 2)


,word,cids1
0,y'all,9.4.2
1,smallness,8.2.6.2
2,human,2
3,human,3.1
4,insides,8.6.4
5,intestines,2.1.8
6,breasts,2.1.8.3
7,breasts,2.1.8.4
8,fear,3.4.2.4
9,grasp,7.3.4.4


In [225]:
#Finding similar words
mean_df1 = pd.DataFrame(columns=['word','similar_word'])
#looping in to find the similar words
for key,value in mean_dict.items():
    mean_df1 = mean_df1.append({'word':key, 'similar_word':value}, ignore_index=True)
print(mean_df1.shape)
mean_df1

(10, 2)


,word,similar_word
0,y'all,gonna
1,smallness,nearness
2,human,humanity
3,insides,innards
4,intestines,trachea
5,breasts,genitals
6,fear,afraid
7,grasp,grip
8,snow,sleet
9,whiteness,white


In [226]:
#mapping all the unmapped words with semantic domain 
sem_dom_unmatch=pd.merge(word_df1,mean_df1,on="word",how="inner")
sem_dom_unmatch.shape

(21, 3)

In [296]:
#Final file merged for the data 
final_merge=pd.merge(merged_data,sem_dom_unmatch,how="left",on="word")
final_merge.shape
final_merge.head()

,index,word,category,cids,cids1,similar_word
0,1,I,Pronouns,9.2.3,NaN,NaN
1,2,thou,NaN,NaN,NaN,NaN
2,2,you,Pronouns,9.2.3,NaN,NaN
3,2,you,Prompters of attention,9.6.3.3,NaN,NaN
4,5,you,Pronouns,9.2.3,NaN,NaN


In [310]:
final_merge.head()
final_data=final_merge.copy()
final_data.head()

,index,word,category,cids,cids1,similar_word
0,0,I,Pronouns,9.2.3,NaN,NaN


In [298]:
final_data.shape
final_data.head()
final_data[final_data['word']=="whiteness"]

,index,word,category,cids,cids1,similar_word
947,175,whiteness,NaN,NaN,1.6.3.1,white
948,175,whiteness,NaN,NaN,2.1.1.1,white
949,175,whiteness,NaN,NaN,2.1.1.5,white
950,175,whiteness,NaN,NaN,2.1.4,white
951,175,whiteness,NaN,NaN,4.1.2,white
952,175,whiteness,NaN,NaN,4.1.9.9,white
953,175,whiteness,NaN,NaN,4.2.6.1.2,white
954,175,whiteness,NaN,NaN,8.3.3.3.1,white


# Language Base dataset

In [ ]:
languages = pd.read_csv('languages.csv')

In [302]:
languages


(776, 777)


,aar_language,index,aba_language,abi_language,abr_language,abs_language,ade_language,adj_language,aeb_language,afo_language,...,yvt_language,zai_language,zbc_language,zlm_language,zmu_language,zne_language,zsm_language,zul_language,zun_language,zyb_language
0,3nu/anu/anú/anə́ː/ínniː,0,m3/mǝ̀/mə̀,ɛ̀mɛ̀,mɪ̃́,beta/bia/béta,nmɩ,Em/ɛ̀m,EnE/آنَا,imi/mi,...,nuya/nuya-ha,na7/naa,akkoh,aku,Natu/ŋad̪u/ŋayi,mi/mì,aku/saya,mina/minne/ngi-,ho7/hon/hoʔ/hoʼ,jau212/kau/kaw/kou
1,3tu/atu/atə́ː/átu,1,f3/fǝ̀/fə̀,ɛ̀fɛ̀,wʊ́,ale/ose/osé,awo/awɔ,ŋ̀,intumE/اِنْتِ,uNo/ŋɔ,...,hiya/hiya-ha,li7/lii,noh,dikəw/mikɯ,5intu/Nintu/yindu/yintu/ŋindu,mo/mò,anda/engkau/kamu/saudara/saudari,u/u-/wena,to7/ton/toʔ/toʼ,muN/nya212
2,ə̀ssəːkə́/ʼusuk,2,,,,,ɛwɔ,,هُوَّ,,...,,laa1be2,,,,ko,dia,u-,,mau33
3,n3nu/nanu,3,el3/èlǝ́/èlə́,ɛ̀rɛ̀,,katoN/katoŋ,Ete/ɛtɩ,,EXnE/اَحْنَا,kinzu,...,waya,laa1du2/laa1nu2/nu,kameh/kitah,kameʔ/kitɤʔ,Nana,,kami/kita,si/si-/th~ina/tini,an/awan/hon,fanxau/raw/tau/tau55/xau/ziu55
4,isin/ìssiːníː/íssiːn,4,éɲə,n-ɛ̂fɛ̀,hʊ̃́,doŋ,ɛyɩ,ɔ̀ɲ,اِنْتُومَا,lu,...,niya-ha,laa1tu2,kam,dikəw sɘmuɯ,n̪uɹ̣a,oni/ànì,engkau/kalian,ni-,ton,xau35


In [304]:
languages.head()
languages.columns

Index(['aar_language', 'index', 'aba_language', 'abi_language', 'abr_language',
       'abs_language', 'ade_language', 'adj_language', 'aeb_language',
       'afo_language',
       ...
       'yvt_language', 'zai_language', 'zbc_language', 'zlm_language',
       'zmu_language', 'zne_language', 'zsm_language', 'zul_language',
       'zun_language', 'zyb_language'],
      dtype='object', length=777)

In [305]:
languages.head()

,aar_language,index,aba_language,abi_language,abr_language,abs_language,ade_language,adj_language,aeb_language,afo_language,...,yvt_language,zai_language,zbc_language,zlm_language,zmu_language,zne_language,zsm_language,zul_language,zun_language,zyb_language
0,3nu/anu/anú/anə́ː/ínniː,0,m3/mǝ̀/mə̀,ɛ̀mɛ̀,mɪ̃́,beta/bia/béta,nmɩ,Em/ɛ̀m,EnE/آنَا,imi/mi,...,nuya/nuya-ha,na7/naa,akkoh,aku,Natu/ŋad̪u/ŋayi,mi/mì,aku/saya,mina/minne/ngi-,ho7/hon/hoʔ/hoʼ,jau212/kau/kaw/kou
1,3tu/atu/atə́ː/átu,1,f3/fǝ̀/fə̀,ɛ̀fɛ̀,wʊ́,ale/ose/osé,awo/awɔ,ŋ̀,intumE/اِنْتِ,uNo/ŋɔ,...,hiya/hiya-ha,li7/lii,noh,dikəw/mikɯ,5intu/Nintu/yindu/yintu/ŋindu,mo/mò,anda/engkau/kamu/saudara/saudari,u/u-/wena,to7/ton/toʔ/toʼ,muN/nya212
2,ə̀ssəːkə́/ʼusuk,2,,,,,ɛwɔ,,هُوَّ,,...,,laa1be2,,,,ko,dia,u-,,mau33
3,n3nu/nanu,3,el3/èlǝ́/èlə́,ɛ̀rɛ̀,,katoN/katoŋ,Ete/ɛtɩ,,EXnE/اَحْنَا,kinzu,...,waya,laa1du2/laa1nu2/nu,kameh/kitah,kameʔ/kitɤʔ,Nana,,kami/kita,si/si-/th~ina/tini,an/awan/hon,fanxau/raw/tau/tau55/xau/ziu55
4,isin/ìssiːníː/íssiːn,4,éɲə,n-ɛ̂fɛ̀,hʊ̃́,doŋ,ɛyɩ,ɔ̀ɲ,اِنْتُومَا,lu,...,niya-ha,laa1tu2,kam,dikəw sɘmuɯ,n̪uɹ̣a,oni/ànì,engkau/kalian,ni-,ton,xau35


In [306]:
final_semdom_data=pd.merge(final_data,languages,on="index",how="inner")
final_semdom_data.shape

(1192, 782)

In [307]:
final_semdom_data.head()

,index,word,category,cids,cids1,similar_word,aar_language,aba_language,abi_language,abr_language,...,yvt_language,zai_language,zbc_language,zlm_language,zmu_language,zne_language,zsm_language,zul_language,zun_language,zyb_language
0,1,I,Pronouns,9.2.3,NaN,NaN,3tu/atu/atə́ː/átu,f3/fǝ̀/fə̀,ɛ̀fɛ̀,wʊ́,...,hiya/hiya-ha,li7/lii,noh,dikəw/mikɯ,5intu/Nintu/yindu/yintu/ŋindu,mo/mò,anda/engkau/kamu/saudara/saudari,u/u-/wena,to7/ton/toʔ/toʼ,muN/nya212
1,2,thou,NaN,NaN,NaN,NaN,ə̀ssəːkə́/ʼusuk,,,,...,,laa1be2,,,,ko,dia,u-,,mau33
2,2,you,Pronouns,9.2.3,NaN,NaN,ə̀ssəːkə́/ʼusuk,,,,...,,laa1be2,,,,ko,dia,u-,,mau33
3,2,you,Prompters of attention,9.6.3.3,NaN,NaN,ə̀ssəːkə́/ʼusuk,,,,...,,laa1be2,,,,ko,dia,u-,,mau33
4,5,you,Pronouns,9.2.3,NaN,NaN,óssoːn/ə́ssəːn/ʼoson,kɛ̀lə́,n-ɛ̂nɛ̀,bɛ́,...,anini,laa1ca1be2,siloh,mikɤʔ/uaŋ diɤʔ,d̪ana,i/àmí,mereka,ba-,ʔāči/∅,za33 mau33


In [308]:
final_semdom_data.to_csv("Final_data_merged.csv",encoding='utf_8_sig')

In [3]:
os.chdir(r"C:\Users\Nikhi\OneDrive - purdue.edu\Mod 3\Industry Practicum\Files")
df=pd.read_csv(r"01272022_Final_data_merged.csv")

C:\Users\Nikhi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,6,347,390,489) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
